In [11]:
from drought.visualization import ee_viz
from drought.data import ee_climate
import ee
import geemap

In [2]:
ee.Initialize()

In [3]:
# ET and PET data for the first 6 months of 2019.
start_date = ee.Date('2019-01-01')
end_date = ee.Date('2019-01-06')\

ic = ee.ImageCollection('MODIS/006/MOD16A2') \
    .select(['ET', 'PET']) \
    .filterDate(start_date, end_date)


In [4]:
# Visualize ET and PET for a single polygon.
ee_viz.viz_single_polygon(ic.first(), polygon_index=7, bands=["ET", "PET"], band_names=['Evapotranspiration','Potential Evepotranspiration'])

Map(center=[-16.482077426075158, -41.27180992107002], controls=(WidgetControl(options=['position', 'transparen…

In [5]:
ee_viz.viz_all_polygons(ic.mean(), bands=["ET", "PET"], band_names=['Evapotranspiration','Potential Evepotranspiration'])

Map(center=[-5.3665973237290725, -54.75046903762467], controls=(WidgetControl(options=['position', 'transparen…

In [9]:
start_date = ee.Date('2001-01-01')
end_date = ee.Date('2019-01-06')

p_monthly = ee_climate.get_monthly_precipitation_data(start_date, end_date)

In [12]:
ee_viz.viz_single_polygon(p_monthly.mean(), polygon_index=7, bands=["precipitation"], band_names=['Mean Monthly Precipitation in mm'], palette=geemap.colormaps.palettes.Blues)

Map(center=[-16.482077426075158, -41.27180992107002], controls=(WidgetControl(options=['position', 'transparen…

In [17]:
ee_viz.viz_all_polygons(p_monthly.mean(), bands=["precipitation"], band_names=['Mean Monthly Precipitation in mm'], palette=geemap.colormaps.palettes.cividis_r)

Map(center=[-5.3665973237290725, -54.75046903762467], controls=(WidgetControl(options=['position', 'transparen…